<a href="https://colab.research.google.com/github/sarajaved797/Walmart-Retail-Sales-Forecasting/blob/master/1_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You are provided with historical sales data for 45 Walmart stores located in different regions. Each store contains a number of departments, and you are tasked with predicting the department-wide sales for each store.

In addition, Walmart runs several promotional markdown events throughout the year. These markdowns precede prominent holidays, the four largest of which are the Super Bowl, Labor Day, Thanksgiving, and Christmas. The weeks including these holidays are weighted five times higher in the evaluation than non-holiday weeks. Part of the challenge presented by this competition is modeling the effects of markdowns on these holiday weeks in the absence of complete/ideal historical data.

stores.csv

This file contains anonymized information about the 45 stores, indicating the type and size of store.

train.csv

This is the historical training data, which covers to 2010-02-05 to 2012-11-01. Within this file you will find the following fields:

Store - the store number
Dept - the department number
Date - the week
Weekly_Sales -  sales for the given department in the given store
IsHoliday - whether the week is a special holiday week
test.csv

This file is identical to train.csv, except we have withheld the weekly sales. You must predict the sales for each triplet of store, department, and date in this file.

features.csv

This file contains additional data related to the store, department, and regional activity for the given dates. It contains the following fields:

Store - the store number
Date - the week
Temperature - average temperature in the region
Fuel_Price - cost of fuel in the region
MarkDown1-5 - anonymized data related to promotional markdowns that Walmart is running. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA.
CPI - the consumer price index
Unemployment - the unemployment rate
IsHoliday - whether the week is a special holiday week
For convenience, the four holidays fall within the following weeks in the dataset (not all holidays are in the data):

Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
Labor Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13



1. Load Data & Basic Setup

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load merged train dataset
df = pd.read_csv('/content/drive/MyDrive/AI ML SQL Excel projects/Walmart Sales Forecasting Time series project/Data/walmart-recruiting-store-sales-forecasting/Prcoessed Data/train_combined.csv', parse_dates=['Date'])



In [22]:
# Basic inspection
print("✅ Dataset loaded!\n")
print("🔹 Shape:", df.shape)
print("\n🔹 Data Types:")
print(df.dtypes)
print("\n🔹 Preview:")
print(df.head())

✅ Dataset loaded!

🔹 Shape: (421570, 16)

🔹 Data Types:
Store                    int64
Dept                     int64
Date            datetime64[ns]
Weekly_Sales           float64
IsHoliday                 bool
Temperature            float64
Fuel_Price             float64
MarkDown1              float64
MarkDown2              float64
MarkDown3              float64
MarkDown4              float64
MarkDown5              float64
CPI                    float64
Unemployment           float64
Type                    object
Size                     int64
dtype: object

🔹 Preview:
   Store  Dept       Date  Weekly_Sales  IsHoliday  Temperature  Fuel_Price  MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5    CPI  Unemployment Type    Size
0      1     1 2010-02-05     24,924.50      False        42.31        2.57        NaN        NaN        NaN        NaN        NaN 211.10          8.11    A  151315
1      1     1 2010-02-12     46,039.49       True        38.51        2.55        NaN      

 Check Missing Values

In [23]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values in Each Column:\n", missing_values)

# Calculate percentage of missing values
missing_percentage = (missing_values / len(df)) * 100
print("\nMissing Value Percentage:\n", missing_percentage)


Missing Values in Each Column:
 Store                0
Dept                 0
Date                 0
Weekly_Sales         0
IsHoliday            0
Temperature          0
Fuel_Price           0
MarkDown1       270889
MarkDown2       310322
MarkDown3       284479
MarkDown4       286603
MarkDown5       270138
CPI                  0
Unemployment         0
Type                 0
Size                 0
dtype: int64

Missing Value Percentage:
 Store           0.00
Dept            0.00
Date            0.00
Weekly_Sales    0.00
IsHoliday       0.00
Temperature     0.00
Fuel_Price      0.00
MarkDown1      64.26
MarkDown2      73.61
MarkDown3      67.48
MarkDown4      67.98
MarkDown5      64.08
CPI             0.00
Unemployment    0.00
Type            0.00
Size            0.00
dtype: float64


Explanation of the Code:
.notnull(): This method returns True for non-null (non-NaN) values and False for NaN values.

For example: If MarkDown1 = 10.5, then MarkDown1.notnull() would return True. If MarkDown1 = NaN, it would return False.

.astype(int): This converts True/False values into 1/0.

True becomes 1, and False becomes 0.

In [24]:
# Create a flag for each MarkDown column (1 if there's a markdown, 0 otherwise)
df['MarkDown1_active'] = df['MarkDown1'].notnull().astype(int)
df['MarkDown2_active'] = df['MarkDown2'].notnull().astype(int)
df['MarkDown3_active'] = df['MarkDown3'].notnull().astype(int)
df['MarkDown4_active'] = df['MarkDown4'].notnull().astype(int)
df['MarkDown5_active'] = df['MarkDown5'].notnull().astype(int)

# Check the new columns
print(df[['MarkDown1', 'MarkDown1_active', 'MarkDown2', 'MarkDown2_active', 'MarkDown3', 'MarkDown3_active']].head())


   MarkDown1  MarkDown1_active  MarkDown2  MarkDown2_active  MarkDown3  MarkDown3_active
0        NaN                 0        NaN                 0        NaN                 0
1        NaN                 0        NaN                 0        NaN                 0
2        NaN                 0        NaN                 0        NaN                 0
3        NaN                 0        NaN                 0        NaN                 0
4        NaN                 0        NaN                 0        NaN                 0


In [25]:
df.sample(5)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,MarkDown1_active,MarkDown2_active,MarkDown3_active,MarkDown4_active,MarkDown5_active
109037,12,8,2012-09-07,"40,637.00",True,88.52,4.12,"17,640.02",84.00,46.18,"3,202.21","2,634.17",130.93,10.93,B,112238,1,1,1,1,1
211375,22,32,2010-09-24,"5,464.13",False,65.14,2.72,NaN,NaN,NaN,NaN,NaN,136.63,8.43,B,119557,0,0,0,0,0
124399,13,54,2011-10-07,77.56,False,60.62,3.59,NaN,NaN,NaN,NaN,NaN,129.69,6.39,A,219622,0,0,0,0,0
125011,13,59,2012-07-27,270.14,False,80.42,3.54,"6,699.60",96.56,5.20,"8,723.87","6,650.26",130.72,5.76,A,219622,1,1,1,1,1
107159,11,94,2010-07-02,"37,796.20",False,83.09,2.67,NaN,NaN,NaN,NaN,NaN,214.55,7.35,A,207499,0,0,0,0,0


Handling Missing Values

In [26]:
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
df[markdown_cols] = df[markdown_cols].fillna(0)


In [27]:
df.sample(5)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,MarkDown1_active,MarkDown2_active,MarkDown3_active,MarkDown4_active,MarkDown5_active
53227,6,33,2010-10-08,"7,127.64",False,65.21,2.63,0.00,0.00,0.00,0.00,0.00,213.27,7.01,A,202505,0,0,0,0,0
27399,3,67,2012-10-05,"1,970.55",False,72.74,3.62,"1,750.80",0.00,3.01,480.24,646.82,226.72,6.03,B,37392,1,0,1,1,1
375587,40,35,2011-08-05,"2,181.64",False,68.10,3.90,0.00,0.00,0.00,0.00,0.00,136.05,4.58,A,155083,0,0,0,0,0
125534,13,72,2011-08-19,"77,559.28",False,76.68,3.58,0.00,0.00,0.00,0.00,0.00,129.24,6.88,A,219622,0,0,0,0,0
106029,11,82,2010-10-08,"20,623.74",False,68.71,2.63,0.00,0.00,0.00,0.00,0.00,215.06,7.56,A,207499,0,0,0,0,0


In [28]:
missing_values=df.isnull().sum()
print(missing_values)

Store               0
Dept                0
Date                0
Weekly_Sales        0
IsHoliday           0
Temperature         0
Fuel_Price          0
MarkDown1           0
MarkDown2           0
MarkDown3           0
MarkDown4           0
MarkDown5           0
CPI                 0
Unemployment        0
Type                0
Size                0
MarkDown1_active    0
MarkDown2_active    0
MarkDown3_active    0
MarkDown4_active    0
MarkDown5_active    0
dtype: int64


Descriptive Stats

Transpose --- for better readability

In [29]:
df.describe().T


,count,mean,min,25%,50%,75%,max,std
Store,"421,570.00",22.20,1.00,11.00,22.00,33.00,45.00,12.79
Dept,"421,570.00",44.26,1.00,18.00,37.00,74.00,99.00,30.49
Date,421570,2011-06-18 08:30:31.963375104,2010-02-05 00:00:00,2010-10-08 00:00:00,2011-06-17 00:00:00,2012-02-24 00:00:00,2012-10-26 00:00:00,NaN
Weekly_Sales,"421,570.00","15,981.26","-4,988.94","2,079.65","7,612.03","20,205.85","693,099.36","22,711.18"
Temperature,"421,570.00",60.09,-2.06,46.68,62.09,74.28,100.14,18.45
Fuel_Price,"421,570.00",3.36,2.47,2.93,3.45,3.74,4.47,0.46
MarkDown1,"421,570.00","2,590.07",0.00,0.00,0.00,"2,809.05","88,646.76","6,052.39"
MarkDown2,"421,570.00",879.97,-265.76,0.00,0.00,2.20,"104,519.54","5,084.54"
MarkDown3,"421,570.00",468.09,-29.10,0.00,0.00,4.54,"141,630.61","5,528.87"
MarkDown4,"421,570.00","1,083.13",0.00,0.00,0.00,425.29,"67,474.85","3,894.53"


In [30]:
df.columns

Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'MarkDown1_active', 'MarkDown2_active', 'MarkDown3_active', 'MarkDown4_active', 'MarkDown5_active'], dtype='object')

Renaming Columns

In [31]:
df.rename(columns={
    'Store': 'store',
    'Dept': 'dept',
    'Date': 'date',
    'Weekly_Sales': 'wk_sales',
    'IsHoliday': 'is_holiday',
    'Temperature': 'temp',
    'Fuel_Price': 'fuel',
    'MarkDown1': 'md1',
    'MarkDown2': 'md2',
    'MarkDown3': 'md3',
    'MarkDown4': 'md4',
    'MarkDown5': 'md5',
    'MarkDown1_active': 'md1_active',
    'MarkDown2_active': 'md2_active',
    'MarkDown3_active': 'md3_active',
    'MarkDown4_active': 'md4_active',
    'MarkDown5_active': 'md5_active',
    'CPI': 'cpi',
    'Unemployment': 'unemp',
    'Type': 'type',
    'Size': 'size'
}, inplace=True)


In [32]:
df.sample(5)

,store,dept,date,wk_sales,is_holiday,temp,fuel,md1,md2,md3,md4,md5,cpi,unemp,type,size,md1_active,md2_active,md3_active,md4_active,md5_active
172649,18,41,2010-12-24,"2,566.51",False,28.16,3.15,0.00,0.00,0.00,0.00,0.00,132.75,9.33,B,120653,0,0,0,0,0
173003,18,44,2012-04-13,"3,586.35",False,47.75,4.03,"5,798.35","9,592.28",11.72,"1,492.20","2,261.14",137.87,8.30,B,120653,1,1,1,1,1
315369,33,11,2010-10-22,94.62,False,74.20,3.01,0.00,0.00,0.00,0.00,0.00,126.38,9.27,A,39690,0,0,0,0,0
345921,36,94,2012-07-20,"32,806.82",False,81.76,3.30,38.04,0.00,0.00,0.00,886.99,220.61,6.62,A,39910,1,0,0,0,1
182402,19,40,2011-08-05,"52,655.08",False,73.84,4.02,0.00,0.00,0.00,0.00,0.00,136.05,7.81,A,203819,0,0,0,0,0


Descriptive Statistics
 is a summary of numeric features to see basic stats (mean, std, min, quartiles, max). This gives an idea of overall distributions.


In [33]:
# Get descriptive statistics for numeric columns (transposed for readability)
print(df.describe().T)



                count                           mean                  min                  25%                  50%                  75%                  max       std
store      421,570.00                          22.20                 1.00                11.00                22.00                33.00                45.00     12.79
dept       421,570.00                          44.26                 1.00                18.00                37.00                74.00                99.00     30.49
date           421570  2011-06-18 08:30:31.963375104  2010-02-05 00:00:00  2010-10-08 00:00:00  2011-06-17 00:00:00  2012-02-24 00:00:00  2012-10-26 00:00:00       NaN
wk_sales   421,570.00                      15,981.26            -4,988.94             2,079.65             7,612.03            20,205.85           693,099.36 22,711.18
temp       421,570.00                          60.09                -2.06                46.68                62.09                74.28               100.14   

Adjusting display settings in Colab

In [34]:


# Set display formatting options for clearer viewing
pd.options.display.float_format = '{:,.2f}'.format  # Format floats with 2 decimal places
pd.set_option('display.max_rows', None)            # Show all rows
pd.set_option('display.max_columns', None)         # Show all columns
pd.set_option('display.width', 2000)                 # Set display width to 2000 characters

# Now when you run your descriptive statistics, the output should be clearer
print(df.describe().T)


                count                           mean                  min                  25%                  50%                  75%                  max       std
store      421,570.00                          22.20                 1.00                11.00                22.00                33.00                45.00     12.79
dept       421,570.00                          44.26                 1.00                18.00                37.00                74.00                99.00     30.49
date           421570  2011-06-18 08:30:31.963375104  2010-02-05 00:00:00  2010-10-08 00:00:00  2011-06-17 00:00:00  2012-02-24 00:00:00  2012-10-26 00:00:00       NaN
wk_sales   421,570.00                      15,981.26            -4,988.94             2,079.65             7,612.03            20,205.85           693,099.36 22,711.18
temp       421,570.00                          60.09                -2.06                46.68                62.09                74.28               100.14   

In [35]:
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0].sort_values(ascending=False))


Series([], dtype: int64)


Understand MarkDown Usage Patterns
 let’s look at how frequently markdowns were used — this can give us insight into promotion strategies, and whether some of these markdowns are even worth keeping in the model.

In [36]:
markdown_flags = ['md1_active', 'md2_active', 'md3_active', 'md4_active', 'md5_active']
print(df[markdown_flags].sum().sort_values(ascending=False))


md5_active    151432
md1_active    150681
md3_active    137091
md4_active    134967
md2_active    111248
dtype: int64


Transporting Data to csv for import

In [37]:
df.head()

,store,dept,date,wk_sales,is_holiday,temp,fuel,md1,md2,md3,md4,md5,cpi,unemp,type,size,md1_active,md2_active,md3_active,md4_active,md5_active
0,1,1,2010-02-05,"24,924.50",False,42.31,2.57,0.00,0.00,0.00,0.00,0.00,211.10,8.11,A,151315,0,0,0,0,0
1,1,1,2010-02-12,"46,039.49",True,38.51,2.55,0.00,0.00,0.00,0.00,0.00,211.24,8.11,A,151315,0,0,0,0,0
2,1,1,2010-02-19,"41,595.55",False,39.93,2.51,0.00,0.00,0.00,0.00,0.00,211.29,8.11,A,151315,0,0,0,0,0
3,1,1,2010-02-26,"19,403.54",False,46.63,2.56,0.00,0.00,0.00,0.00,0.00,211.32,8.11,A,151315,0,0,0,0,0
4,1,1,2010-03-05,"21,827.90",False,46.50,2.62,0.00,0.00,0.00,0.00,0.00,211.35,8.11,A,151315,0,0,0,0,0


In [39]:
# Save the cleaned DataFrame to a CSV in a data folder
df.to_csv('/content/drive/MyDrive/AI ML SQL Excel projects/Walmart Sales Forecasting Time series project/notebooks/cleaned_walmart_sales.csv', index=False)
